In [3]:
import pandas as pd
from fonction import Orcid_Researcher
import requests
import os

In [11]:
hal_df = pd.read_excel(r"ressources\data_humber_hal.xlsx")
scopus_df = pd.read_excel(r"ressources\scopus_mh.xlsx")
orcid_df = pd.read_excel(r"ressources\orcid_publication.xlsx")

In [166]:
def base_link(prefix: str = None, query:str = None):
    """" This founction performs searches basics such as author, simple search
    
    Args : 
        prefix(str) : domain name
        query(str) : search information, parameter can be used. 
    """
    # A documenter !!!!!!
    root = f"https://api.archives-ouvertes.fr/{"ref/" + prefix 
                                            if prefix in ["anrproject","doctype","instance","metadata","structure","metadatalist","journal","domain","europeanproject","author"] 
                                            else "search/"}"
    endpoint = f"?q={query}&wt=json"
    return requests.get(root+endpoint).json()

def id_author(lastName:str, firstName:str):
    """ This founction will search for the IDs of the authors you want.
    Args :
        lastName(str) : Nom de l'auteur
        firstName(str) : prénom de l'auteur
    """
    if not lastName:
        return "Erreur Recherche HAL : Veuillez renseigner le nom de famille."
    if not firstName:
        return "Erreur Recherche HAL : Veuillez renseigner le prénom."
    
    req = base_link(prefix = "author", query= f"{lastName} {firstName}")

    id_auth = []
    docs = req["response"]["docs"]
    numFound = req["response"]["numFound"]
        
    if int(numFound) == 0:
        return "Erreur Recherche HAL : Nous n'avons pas trouvé de données\nVérifiez les informations renseignées."
    
    for doc in docs:
        docid = doc["docid"]
        if docid.split("-")[1] not in id_auth:
            id_auth.append(docid) # .split("-")[1]
    return id_auth

def get_hal_researcher_data(lastName:str,firstName:str):
    """ This founction will search for the IDs of the authors you want.
    Args :
        lastName(str) : Nom de l'auteur
        firstName(str) : prénom de l'auteur
    """

    # authIdForm_i:158428 -> Marc Humbert -> 449
    ids = id_author(lastName = lastName, firstName=firstName)
    if isinstance(ids, list):
        data_hal = []
        for id in ids:
            req = base_link(query=f"authIdFormPerson_s:{id}&fl=title_s,doiId_s,authIdHal_s,pubmedId_id,journalTitle_s,journalPublisher_s,publicationDate_s,authLastNameFirstName_s&start=0&rows=1000")
            print(req)
            docs = req["response"]["docs"]
            for doc in docs:
                title_s = doc["title_s"][0]
                journalTitle_s = doc.get("journalTitle_s", None)
                journalPublisher_s = doc.get("journalPublisher_s", None)
                publicationDate_s = doc["publicationDate_s"]
                doiId_s = doc.get("doiId_s", None)
                pubmedId_id = doc.get("pubmedId_id", None)
                authLastNameFirstName_s = doc.get("authLastNameFirstName_s", None)
                data_hal.append({
                    "Titre_journal": journalTitle_s,
                    "Author Full Names" : authLastNameFirstName_s,
                    "Title_article": title_s,
                    "journalPublisher_s" : journalPublisher_s,
                    "publicationDate_s" : publicationDate_s,
                    "DOI": doiId_s,
                    "pubmedId": pubmedId_id
                })
    
    # def test(liste):
    #     for i in liste:
    #         if lastName in i:
    #             return liste
    
    # df_hal["Author Full Names"] = df_hal["Author Full Names"].apply(test)
    
    return pd.DataFrame(data_hal)

In [169]:
fabrice = get_hal_researcher_data("André", "Fabrice") Besse Benjamin

{'response': {'numFound': 131, 'start': 0, 'maxScore': 7.7248535, 'numFoundExact': True, 'docs': [{'title_s': ['Outcomes in oncogenic-addicted advanced NSCLC patients with actionable mutations identified by liquid biopsy genomic profiling using a tagged amplicon-based NGS assay'], 'journalTitle_s': 'PLoS ONE', 'journalPublisher_s': 'Public Library of Science', 'authLastNameFirstName_s': ['Remon Jordi', 'Swalduz Aurelie', 'Planchard David', 'Ortiz-Cuaran Sandra', 'Mezquita Laura', 'Lacroix Ludovic', 'Jovelet Cecile', 'Rouleau Etienne', 'Leonce Camille', 'de Kievit Frank', 'Morris Clive', 'Jones Greg', 'Mercier Kelly', 'Howarth Karen', 'Green Emma', 'Pérol Maurice', 'Saintigny Pierre', 'Besse Benjamin'], 'authIdHal_s': ['pierre-saintigny'], 'doiId_s': '10.1371/journal.pone.0234302', 'publicationDate_s': '2020-06-11'}, {'title_s': ['Focus on Recommendations for the Management of Non-small Cell Lung Cancer'], 'journalTitle_s': 'CardioVascular and Interventional Radiology', 'journalPublishe

In [170]:
fabrice

,Titre_journal,Author Full Names,Title_article,journalPublisher_s,publicationDate_s,DOI,pubmedId
0,PLoS ONE,"[Remon Jordi, Swalduz Aurelie, Planchard David...",Outcomes in oncogenic-addicted advanced NSCLC ...,Public Library of Science,2020-06-11,10.1371/journal.pone.0234302,None
1,CardioVascular and Interventional Radiology,"[Lavaud Pernelle, Besse Benjamin, de Baere Thi...",Focus on Recommendations for the Management of...,Cardiovascular and Interventional Radiological...,2019-05-06,10.1007/s00270-019-02222-9,None
2,Clinical and Translational Radiation Oncology,"[Levy Antonin, Khalifa Jonathan, Martin Etienn...",Stereotactic body radiotherapy for extra-crani...,Elsevier,2023-07,10.1016/j.ctro.2023.100637,None
3,Journal of Thoracic Oncology,"[Malhotra Jyoti, Nikolinakos Petros, Leal Tici...",A Phase 1–2 Study of Rovalpituzumab Tesirine i...,"Lippincott, Williams & Wilkins",2021-09,10.1016/j.jtho.2021.02.022,None
4,Annals of Diagnostic Pathology,"[Thomas de Montpréville Vincent, Lacroix Ludov...",Non-small cell lung carcinomas with CTNNB1 (be...,WB Saunders,2020-06,10.1016/j.anndiagpath.2020.151522,None
...,...,...,...,...,...,...,...
144,Cancer Discovery,"[Goubet Anne-Gaëlle, Lordello Leonardo, Alves ...",Escherichia coli –Specific CXCL13-Producing TF...,American Association for Cancer Research,2022-10-05,10.1158/2159-8290.CD-22-0201,None
145,Science,"[Fidelle Marine, Rauber Conrad, Alves Costa Si...",A microbiota-modulated checkpoint directs immu...,American Association for the Advancement of Sc...,2023-06-09,10.1126/science.abo2296,None
146,Clinical Cancer Research,"[Dall'Olio Filippo, Zrafi Wael, Roelants Veron...",Metabolic tumor volume assessed by 18F FDG - P...,American Association for Cancer Research,2024-10-22,10.1158/1078-0432.CCR-24-1993,None
147,JCO precision oncology,"[Hadoux Julien, Al Ghuzlan Abir, Lamartina Liv...",Patterns of Treatment Failure After Selective ...,American Society of Clinical Oncology,2023-09,10.1200/PO.23.00053,None


In [156]:
fab = fabrice.copy()
a = fab["Author Full Names"][0]
fab["Author Full Names"] = fab["Author Full Names"].apply(test)

In [157]:
fab

,Titre_journal,Author Full Names,Title_article,journalPublisher_s,publicationDate_s,DOI,pubmedId
0,International Journal of Clinical Practice,"[O'Callaghan Dermot, Savale Laurent, Magnier R...",Current management approaches to portopulmonar...,Wiley,2010-12-22,10.1111/j.1742-1241.2010.02600.x,None
1,None,"[Lambin Xavier, Humbert Marc]",Distance Exams ? Can targeted warnings discour...,None,2023,None,None
2,Respiratory Medicine,"[Devillier Philippe, Humbert Marc, Boye Alain,...",Efficacy and safety of once-daily fluticasone ...,Elsevier,2018-08,10.1016/j.rmed.2018.06.009,None
3,Seminars in Respiratory and Critical Care Medi...,"[Lechartier Benoit, Kularatne Mithum, Jaïs Xav...",Updated Hemodynamic Definition and Classificat...,Thieme Publishing,2023-08-18,10.1055/s-0043-1770115,None
4,Cardiovascular Research,"[Lambert Mélanie, Boet Angèle, Rucker-Martin C...",Loss of KCNK3 is a hallmark of RV hypertrophy/...,Oxford University Press (OUP),2018-05-01,10.1093/cvr/cvy016,None
...,...,...,...,...,...,...,...
451,European Respiratory Journal,"[Taillé Camille, Chanez Pascal, Devouassoux Gi...",Mepolizumab in a population with severe eosino...,European Respiratory Society,2020,10.1183/13993003.02345-2019,None
452,Cells,"[Leuillier Matthieu, Platel Valentin, Tu Ly, F...",Inhibition of Soluble Epoxide Hydrolase Does N...,MDPI,2023-02-20,10.3390/cells12040665,None
453,American Journal of Respiratory and Critical C...,"[Jutant Etienne-Marie, Jaïs Xavier, Girerd Bar...",Phenotype and Outcomes of Pulmonary Hypertensi...,American Thoracic Society,2020-09-15,10.1164/rccm.202001-0105OC,None
454,The Lancet Respiratory Medicine,"[Tharaux Pierre-Louis, Pialoux Gilles, Pavot A...",Effect of anakinra versus usual care in adults...,Elsevier,2021-03,10.1016/S2213-2600(20)30556-7,None


In [42]:
import matplotlib.pyplot as plt
import numpy as np

y = np.array([69.5, 30.5])
# myexplode = [0.05, 0]

# mylabels = ["in", "not in"] if y[0] > 50 else ["not in", "in"]
# colors = ["green", "red"] if y[0] > 50 else ["red", "green"]

# plt.pie(y, 
# labels = mylabels,
# autopct = "%1.1f%%", 
# explode = myexplode, 
# colors = colors)
# plt.show() 


# Configuration des styles pour chaque catégorie
config = {
    "Dans Orcid": {"color": "green", "explode": 0.05},
    "Pas dans Orcid": {"color": "red", "explode": 0}
}

def create_pie_chart(values):
    # Déterminer la catégorie dominante
    dominant_category = "Dans" if values[0] > 50 else "not in"
    categories = [dominant_category, "not in" if dominant_category == "in" else "in"]
    
    # Créer les listes de paramètres dans le bon ordre
    colors = [config[cat]["color"] for cat in categories]
    explode = [config[cat]["explode"] for cat in categories]
    
    # Créer et afficher le graphique
    plt.pie(values, 
            labels=categories,
            colors=colors,
            explode=explode,
            autopct="%1.1f%%")
    plt.show()

# Utilisation
y = np.array([49.5, 50.5])
create_pie_chart(y)

KeyError: 'not in'